In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import json
from src.train import *
from pathlib import Path

In [ ]:
min_count = 10
windows = [2, 5, 10]
vector_sizes = [50, 100, 500]
languages = ['bxr', 'myv', 'kv']

In [ ]:
language = languages[2]

In [ ]:
# path = Path("./embeddings/"+str(vector_size)+"/"+str(window))
# wes = list(path.iterdir())
# for we in wes:
#     we.unlink()
# path.rmdir()
path.mkdir(parents=True)

In [ ]:
wiki2text(
    "./data/"+language+'wiki.xml.bz2', 
    "./data/"+language+'.txt', 
    article_min_tokens=0,
    token_min_len=0,
    token_max_len=5000
)

In [ ]:
for vector_size in vector_sizes:
    for window in windows:
        print("vector_size:", vector_size, "window:", window)
        pt = "./embeddings/"+language+"/"+str(vector_size)+"/"+str(window)+"/"
        w2v(vector_size, window, min_count, fname=pt+"cbow", fcorpus="./data/"+language+".txt", sg=0)
        w2v(vector_size, window, min_count, fname=pt+"sg", fcorpus="./data/"+language+".txt", sg=1)
#         ft(vector_size, window, min_count, fname=pt+"ft", fcorpus="./data/"+language+".txt")

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

for language in languages:
    fname = str(Path("./data/"+language+".txt").absolute())
    for vector_size in vector_sizes:
        for window in windows:
            pt = "./embeddings/"+language+"/"+str(vector_size)+"/"+str(window)+"/"
            wef = "/tmp/"+str(vector_size)+str(window)
            !glove/vocab_count -min-count $min_count -verbose 2 < $fname > /tmp/vocab.txt
            !glove/cooccur -memory 4.0 -vocab-file /tmp/vocab.txt -verbose 2 -window-size $window  < $fname > /tmp/cooccurrence
            !glove/shuffle -memory 4.0 -verbose 2 < /tmp/cooccurrence > /tmp/cooccurrence.shuf
            !glove/glove -save-file $wef -threads 8 -input-file /tmp/cooccurrence.shuf -x-max 10 -iter 20 -vector-size $vector_size -binary 0 -vocab-file /tmp/vocab.txt -verbose 2

            glove_file = datapath(wef+".txt")
            tmp_file = datapath(pt+"glove")
            _ = glove2word2vec(glove_file,pt+"glove")

In [ ]:
from src.pmi import *


methods = ['PpmiWeighting', 'EpmiWeighting', 'PlmiWeighting', 'PlogWeighting']

for language in languages:
    print(language)
    x = []
    with open("./data/"+language+".txt") as file:
        for l in file:
            x.append(l.strip())

    for vector_size in [0]:
        for method in methods:
            for window in windows:
                a = build_pairs(x, window)
                prepare_pmi(a)
                pmi(method, vector_size)
                convert("/tmp/space.dm", "./embeddings/"+language+"/"+str(vector_size)+"/"+str(window)+"/"+method)